# Imports


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
from time import time
from sp_sims.simulators.stochasticprocesses import BDStates
from samprecon.environments.OneEpisodeEnvironments import MarkovianUniformEnvironment
from samprecon.samplers.agents import SimpleAgent 
from samprecon.reconstructors.NNReconstructors import RNNReconstructor
from tqdm.notebook import tqdm

plt.style.use('rose-pine-dawn')
rnd.seed(int(time()))
%load_ext autoreload
%autoreload 2

In [2]:
# Generate Environments on which to learn 
high_res_delta = 1e-1 # For generating the dataset and later sample
baseline_rates = {"lam": 1/10,"mu":4/10}
epochs = 100
lenth_of_episode = 12
step_path_length = 1
sampling_budget = 4
used_path_length = 64 # So that we can let the process reach stationarity and take samples from stationary distribution
num_states = 4
avg_span = np.mean(1/np.array(
            list(baseline_rates.values())
    ))
max_decimation = (avg_span/high_res_delta)*4 # Max decimation factor #CHECK: Maybe not divide by 2
current_decimation_factor = int(# We can start somewhere in between 
    avg_span//high_res_delta)
print(f"Decimation factor is {current_decimation_factor}")
# Set random seed with time for randomnessj

Decimation factor is 62


In [3]:
# Initialize context first
stateGen = BDStates(baseline_rates, high_res_delta, num_states)
sampling_arbiter = SimpleAgent(num_states,max_decimation)
reconstructor = RNNReconstructor(amnt_states = num_states)

env = MarkovianUniformEnvironment(
    state_generator = stateGen,
    sampling_arbiter = sampling_arbiter,
    reconstructor = reconstructor, 
    starting_decrate = current_decimation_factor,
)

In [4]:
ebar = tqdm(range(epochs), desc = "Epochs",leave=False, position=0)
e_losses = []
for epoch in range(epochs):
    # We generate a single step from the generator process
    sbar = tqdm(range(lenth_of_episode), desc = "Steps", leave=True, position=1)
    for step in range(lenth_of_episode):

        loss = env.step() 
        sbar.set_description(f"Loss: {loss}")

        sbar.update(1)
    e_losses = np.append(e_losses,loss)
    ebar.set_description(f"Epoch Mean Loss: {np.mean(e_losses[:-10])}")
    # We get reward based on how close we got to maximum information 

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Steps:   0%|          | 0/12 [00:00<?, ?it/s]

/Users/rac716/Research/SamplingReconstruction/samprecon/samplers/spatial_transformers.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(decimation_intervals[i])
2023-12-06 15:14:49,861 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:49,862 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:49,912 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:49,914 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:49,965 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:49,968 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:50,029 - Markov

Steps:   0%|          | 0/12 [00:00<?, ?it/s]

2023-12-06 15:14:50,510 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:50,511 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:50,564 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:50,565 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:50,619 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:50,619 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:50,668 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:50,669 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:14:50,719 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:14:50,720 - MarkovianUniformEn

...

Steps:   0%|          | 0/12 [00:00<?, ?it/s]

2023-12-06 15:15:02,611 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:02,612 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:02,661 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:02,662 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:02,711 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:02,712 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:02,766 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:02,767 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:02,819 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:02,820 - MarkovianUniformEn

Steps:   0%|          | 0/12 [00:00<?, ?it/s]

2023-12-06 15:15:03,246 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:03,247 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:03,297 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:03,298 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:03,349 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:03,350 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:03,399 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:03,400 - MarkovianUniformEnvironment - INFO - Sum of weight difference  reconstructor0.0000
2023-12-06 15:15:03,452 - MarkovianUniformEnvironment - INFO - Sum of weight difference arbiterer0.0000
2023-12-06 15:15:03,453 - MarkovianUniformEn

KeyboardInterrupt: 